# 人脸检测和识别推理流程

以下示例展示了如何使用facenet_pytorch python包，在使用在VGGFace2数据集上预训练的Inception Resnet V1模型上对图像数据集执行人脸检测和识别。

以下PyTorch方法已包含：

* 数据集
* 数据加载器
* GPU/CPU处理

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

#### 判断是否有nvidia GPU可用

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('在该设备上运行: {}'.format(device))

在该设备上运行: cpu


#### 定义MTCNN模块

为了说明，默认参数已显示，但不是必需的。请注意，由于MTCNN是一组神经网络和其他代码，因此必须以以下方式传递设备，以便在需要内部复制对象时启用。

查看`help(MTCNN)`获取更多细节。

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### 定义Inception Resnet V1模块

设置classify=True以使用预训练分类器。对于本示例，我们将使用该模型输出嵌入/卷积特征。请注意，在推理过程中，将模型设置为`eval`模式非常重要。

查看`help(InceptionResnetV1)`获取更多细节。

In [6]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

#### 定义数据集和数据加载器

我们向数据集添加了`idx_to_class`属性，以便稍后轻松重编标签索引为身份名称。

In [7]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('../data/test_images')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

#### 执行MTCNN人脸检测

迭代DataLoader对象并检测每个人脸及其关联的检测概率。如果检测到脸部，`MTCNN`的前向方法将返回裁剪到检测到的脸部的图像。默认情况下，仅返回检测到的单个面部-要使`MTCNN`返回所有检测到的面部，请在上面创建MTCNN对象时设置`keep_all=True`。

要获取边界框而不是裁剪的人脸图像，可以调用较低级别的`mtcnn.detect()`函数。查看`help(mtcnn.detect)`获取详细信息。

In [29]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('检测到的人脸及其概率: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

检测到的人脸及其概率: 0.999983
检测到的人脸及其概率: 0.999934
检测到的人脸及其概率: 0.999733
检测到的人脸及其概率: 0.999876
检测到的人脸及其概率: 0.999992


#### 计算图像嵌入

MTCNN将返回所有面部图像的相同大小，从而可以使用Resnet识别模块轻松进行批处理。在这里，由于我们只有一些图像，因此我们构建一个单个批次并对其执行推理。

对于实际数据集，代码应修改为控制传递给Resnet的批处理大小，特别是如果在GPU上处理。对于重复测试，最好将人脸检测（使用MTCNN）与嵌入或分类（使用InceptionResnetV1）分开，因为剪切面或边界框的计算可以一次执行，检测到的面部可以保存供将来使用。

In [30]:
aligned = torch.stack(aligned).to(device)
print(aligned)
embeddings = resnet(aligned).detach().cpu()

tensor([[[[-0.4883, -0.5352, -0.5195,  ...,  0.8633,  0.8633,  0.8633],
          [-0.5664, -0.5430, -0.4492,  ...,  0.8633,  0.8711,  0.8711],
          [-0.5508, -0.5273, -0.3633,  ...,  0.8711,  0.8711,  0.8711],
          ...,
          [-0.5352, -0.5273, -0.4727,  ..., -0.0586, -0.0898, -0.1523],
          [-0.3867, -0.3320, -0.2539,  ..., -0.0508, -0.0820, -0.1523],
          [-0.2852, -0.2773, -0.2773,  ..., -0.0586, -0.1055, -0.1523]],

         [[-0.6367, -0.6836, -0.6758,  ...,  0.7617,  0.7617,  0.7617],
          [-0.6992, -0.6836, -0.6289,  ...,  0.7617,  0.7695,  0.7695],
          [-0.6836, -0.6680, -0.5742,  ...,  0.7695,  0.7695,  0.7695],
          ...,
          [-0.7148, -0.7070, -0.6680,  ..., -0.4336, -0.4570, -0.5039],
          [-0.6211, -0.5742, -0.5117,  ..., -0.4258, -0.4570, -0.5117],
          [-0.5586, -0.5508, -0.5508,  ..., -0.4258, -0.4727, -0.5117]],

         [[-0.7539, -0.7773, -0.7852,  ...,  0.6914,  0.6914,  0.6914],
          [-0.7773, -0.7695, -

#### 打印各类别的距离矩阵

In [10]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
pd.DataFrame(dists, columns=names, index=names)

,angelina_jolie,bradley_cooper,kate_siegel,paul_rudd,shea_whigham
angelina_jolie,0.000000,1.447480,0.887728,1.434377,1.399073
bradley_cooper,1.447480,0.000000,1.313749,1.011980,1.038684
kate_siegel,0.887728,1.313749,0.000000,1.388992,1.379655
paul_rudd,1.434377,1.011980,1.388992,0.000000,1.104095
shea_whigham,1.399073,1.038684,1.379655,1.104095,0.000000
